In [1]:
import pandas as pd
import http.client
import requests
import json
import time
from datetime import datetime
import xlsxwriter

#---------------connectETH--------------------------------------------------------

conn = http.client.HTTPSConnection("www.mercadobitcoin.net")
payload = ""
conn.request("GET", "/api/ETH/ticker/", payload)

res = conn.getresponse()
data = res.read()
data = data.decode("utf-8").replace('{"ticker": {','').replace('}}','').replace('"','').split('\n')
data = data[0].split(",")

nome = data[0].split(":")[0].split(",")[0]
valor = data[0].split(":")[1].split(",")[0]

CotacaoETH = {}
for row in data:
    nome = row.split(":")[0].split(",")[0]
    valor = row.split(":")[1].split(",")[0]
    CotacaoETH[nome] = float(valor)

date = int(CotacaoETH['date'])
date = datetime.fromtimestamp(int(date)).strftime('%d/%m/%Y')

dfETH = pd.json_normalize(CotacaoETH) 
dfETH['Data'] = date
dfETH['Hora'] = datetime.fromtimestamp(int(CotacaoETH['date'])).strftime('%H:%M:%S')
dfETH['nome'] = "ETHER"
dfETH.drop(["date"], axis=1, inplace=True)

#-----------------------------------------------------------------------
#---------------connectBTC--------------------------------------------------------
conn.request("GET", "/api/BTC/ticker/", payload)
res = conn.getresponse()

dataBTC = res.read()
dataBTC = dataBTC.decode("utf-8").replace('{"ticker": {','').replace('}}','').replace('"','').split('\n')
dataBTC = dataBTC[0].split(",")

nomeBTC = dataBTC[0].split(":")[0].split(",")[0]
valorBTC = dataBTC[0].split(":")[1].split(",")[0]

CotacaoBTC = {}
for row in dataBTC:
    nomeBTC = row.split(":")[0].split(",")[0]
    valorBTC = row.split(":")[1].split(",")[0]
    CotacaoBTC[nomeBTC] = float(valorBTC)
    
date = int(CotacaoETH['date'])
date = datetime.fromtimestamp(int(date)).strftime('%d/%m/%Y')

dfBTC = pd.json_normalize(CotacaoBTC) 
dfBTC['Data'] = date
dfBTC['Hora'] = datetime.fromtimestamp(int(CotacaoBTC['date'])).strftime('%H:%M:%S')
dfBTC['nome'] = "BITCOIN"
dfBTC.drop(["date"], axis=1, inplace=True)

price = pd.merge(dfETH, dfBTC, how = 'outer')

with pd.ExcelWriter("CTC.xlsx", mode="a", engine="openpyxl", if_sheet_exists="COTACAO_CRIPTO",) as writer:
    price.to_excel(writer, sheet_name="COTACAO_CRIPTO")
    
tabela = pd.read_excel(open('CTC.xlsx', 'rb'), sheet_name='COTACAO_CRIPTO') 

price

,high,low,vol,last,buy,sell,open,Data,Hora,nome
0,5930.046040,5656.52,351.946690,5842.963510,5842.96351,5849.969990,5731.8,13/07/2022,08:00:50,ETHER
1,108554.385786,105000.00,44.598442,107952.195323,107800.00000,107895.716427,106000.0,13/07/2022,08:00:52,BITCOIN
